# ENEI-2022: Data wrangling

In [ ]:
import os
import json
import pandas as pd
from typing import Tuple
from janitor import clean_names

pd.set_option("display.max_columns", None)

## Import data

In [ ]:
def load_data(path: str) -> Tuple[pd.DataFrame]:
    survey = pd.read_excel(os.path.join(path, "enei.xlsx"))
    features = pd.read_excel(os.path.join(path, "dict.xlsx"), skiprows=4).iloc[:318, :]
    factors = pd.read_excel(os.path.join(path, "dict.xlsx"), skiprows=326)
    return clean_names(survey), clean_names(features), clean_names(factors)

survey_raw, features_raw, factors_raw = load_data(path=os.path.join(os.getcwd(), "raw"))

## Filter surveys by the proportion of nans in columns

In [ ]:
def remove_columns_with_nans(df, beta):
    nan_proportion = df.isna().mean()
    columns_to_drop = nan_proportion[nan_proportion >= beta].index
    df_filtered = df.drop(columns=columns_to_drop)
    return df_filtered

survey_filtered = remove_columns_with_nans(survey_raw, 0.2).dropna(axis=0)
survey_filtered

## Take a look: what features are we dropping?

In [ ]:
features = (
    features_raw
    .assign(
        nombre_de_la_variable = lambda x: x.nombre_de_la_variable.str.lower(),
        etiqueta_de_la_variable = lambda x: x.etiqueta_de_la_variable.str.lower()
    )
    .drop('posicion', axis=1)
)

features_description = dict(zip(features['nombre_de_la_variable'], features['etiqueta_de_la_variable']))

In [ ]:
filtered_features_description = {key: value for key, value in features_description.items() if key in list(survey_filtered.columns)}
dropped_features_description = {key: value for key, value in features_description.items() if key not in list(survey_filtered.columns)}

names_and_dicts = {
    'filtered_features_description': filtered_features_description,
    'dropped_features_description': dropped_features_description
}

path = os.path.join(os.getcwd(), "features")
os.makedirs(path, exist_ok=True)

for name, dictionary in names_and_dicts.items():
    with open(os.path.join(os.getcwd(), "features", f"{name}.json"), "w") as file:
        json.dump(dictionary, file, ensure_ascii=False, indent=4)

## I still need a way to map label encodings with factor levels

In [ ]:
survey_raw

In [ ]:
factors = (
    factors_raw
    .assign(nombre_de_la_variable = lambda x: x.nombre_de_la_variable.ffill())
    .assign(
        nombre_de_la_variable = lambda x: x.nombre_de_la_variable.str.lower(),
        etiqueta = lambda x: x.etiqueta.str.lower()
    )
)

factors

In [ ]:
features_with_mapped_levels = []

for feature_name in factors.nombre_de_la_variable.unique():
    try:
        feature_levels_df = (
            factors
            .query(f"nombre_de_la_variable == '{feature_name}'")
            .drop("nombre_de_la_variable", axis=1)
        )
        feature_levels_dict = dict(feature_levels_df.set_index('valor')['etiqueta'])
        mapped_feature = (
            survey_raw[f"{feature_name}"].map(feature_levels_dict)
        )
        features_with_mapped_levels.append(mapped_feature)
    except KeyError:
        pass

survey_with_levels = pd.concat(features_with_mapped_levels, axis=1)

In [ ]:
survey_with_levels